In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime       #To access datetime
from pandas import Series           #To work on series
import warnings                     #To ignote the warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv("Train_SU63ISt.csv")
test  = pd.read_csv("Test_0qrQsBZ.csv")

In [3]:
# Realizo una copia de los datos de entrenamiento y prueba para que,
# si realizo cambios en este conjunto de datos, no pierda
# el conjunto de datos original

train_original = train.copy()
test_original  = test.copy()

In [4]:
train.columns

Index(['ID', 'Datetime', 'Count'], dtype='object')

In [5]:
print(test.columns)

Index(['ID', 'Datetime'], dtype='object')


In [6]:
train.head(4)

,ID,Datetime,Count
0,0,25-08-2012 00:00,8
1,1,25-08-2012 01:00,2
2,2,25-08-2012 02:00,6
3,3,25-08-2012 03:00,2


In [7]:
test.head(4)

,ID,Datetime
0,18288,26-09-2014 00:00
1,18289,26-09-2014 01:00
2,18290,26-09-2014 02:00
3,18291,26-09-2014 03:00


El archivo de entrenamiento  tiene Id, Datetime y Count, mientras que el de prueba únicamente tiene ID y Datetime, es decir, **se tiene que predecir el recuento de pasajareros para el archivo prueba** 

In [8]:
train.dtypes

ID           int64
Datetime    object
Count        int64
dtype: object

In [9]:
test.dtypes

ID           int64
Datetime    object
dtype: object

In [10]:
train.shape

(18288, 3)

In [11]:
test.shape

(5112, 2)

Como el tipo de dato de **Datetime**  es **objeto**, entonces, antes que nad, se tiene que cambiar el tipo de dato al formato de fecha y hora, de lo contrario no se puede extraer caracterísitca de él. 

In [12]:
train['Datetime']          = pd.to_datetime(train.Datetime,format='%d-%m-%Y %H:%M') 
test['Datetime']           = pd.to_datetime(test.Datetime,format='%d-%m-%Y %H:%M')
test_original['Datetime']  = pd.to_datetime(test_original.Datetime,format='%d-%m-%Y %H:%M')
train_original['Datetime'] = pd.to_datetime(train_original.Datetime,format='%d-%m-%Y %H:%M')

Extraigo el año, mes, día y hora de Datetime para validar las hipótesis. 
Añado los valores de año, mes, dia y hora a cada conjunto de datos a partir de Datetime

In [13]:
for i in (train, test, test_original, train_original):
    i['year']  = i.Datetime.dt.year 
    i['month'] = i.Datetime.dt.month 
    i['day']   = i.Datetime.dt.day
    i['Hour']  = i.Datetime.dt.hour 

In [14]:
train.columns

Index(['ID', 'Datetime', 'Count', 'year', 'month', 'day', 'Hour'], dtype='object')

In [15]:
train.dtypes

ID                   int64
Datetime    datetime64[ns]
Count                int64
year                 int64
month                int64
day                  int64
Hour                 int64
dtype: object

In [16]:
train.head()

,ID,Datetime,Count,year,month,day,Hour
0,0,2012-08-25 00:00:00,8,2012,8,25,0
1,1,2012-08-25 01:00:00,2,2012,8,25,1
2,2,2012-08-25 02:00:00,6,2012,8,25,2
3,3,2012-08-25 03:00:00,2,2012,8,25,3
4,4,2012-08-25 04:00:00,2,2012,8,25,4


Dentro de las hipótesis del patrón de tráfico se siente que entre semana habrá mayor tráfico que los fines de semana, **Entonces, hagamos una variable de fin de semana para visualizar el impacto del fin de semana en el tráfico**

- Primero extraigo el día de la semana de Datetime y luego en función de los valores que asigne si el día es fin de semana o no.

- Los valores 5 y 6 representan que los días son fines de semana

In [17]:
train['day of week'] = train['Datetime'].dt.dayofweek
temp  =  train['Datetime']

**Asigno un 1 si el día es fín de semana, 0 si no lo es**

In [18]:
def applyer(row):
    
    if row.dayofweek == 5 or row.dayofweek == 6:
        return 1
    else:
        return 0
    
temp2 = train['Datetime'].apply(applyer) 
train['weekend'] = temp2

In [19]:
train.head()

,ID,Datetime,Count,year,month,day,Hour,day of week,weekend
0,0,2012-08-25 00:00:00,8,2012,8,25,0,5,1
1,1,2012-08-25 01:00:00,2,2012,8,25,1,5,1
2,2,2012-08-25 02:00:00,6,2012,8,25,2,5,1
3,3,2012-08-25 03:00:00,2,2012,8,25,3,5,1
4,4,2012-08-25 04:00:00,2,2012,8,25,4,5,1


In [20]:
train.index = train['Datetime'] # Indexo Datetiem para objeter el período de tiempo 
                                # en el eje x

df = train.drop('ID', 1)        # Elimino la variable ID para dejar Datetime en el eje x
                                # el 1 especifica que se elimina toda la columna
ts = df['Count']

In [21]:
df.head()

,Datetime,Count,year,month,day,Hour,day of week,weekend
Datetime,,,,,,,,
2012-08-25 00:00:00,2012-08-25 00:00:00,8,2012,8,25,0,5,1
2012-08-25 01:00:00,2012-08-25 01:00:00,2,2012,8,25,1,5,1
2012-08-25 02:00:00,2012-08-25 02:00:00,6,2012,8,25,2,5,1
2012-08-25 03:00:00,2012-08-25 03:00:00,2,2012,8,25,3,5,1
2012-08-25 04:00:00,2012-08-25 04:00:00,2,2012,8,25,4,5,1


In [22]:
train.head()

,ID,Datetime,Count,year,month,day,Hour,day of week,weekend
Datetime,,,,,,,,,
2012-08-25 00:00:00,0,2012-08-25 00:00:00,8,2012,8,25,0,5,1
2012-08-25 01:00:00,1,2012-08-25 01:00:00,2,2012,8,25,1,5,1
2012-08-25 02:00:00,2,2012-08-25 02:00:00,6,2012,8,25,2,5,1
2012-08-25 03:00:00,3,2012-08-25 03:00:00,2,2012,8,25,3,5,1
2012-08-25 04:00:00,4,2012-08-25 04:00:00,2,2012,8,25,4,5,1


In [23]:
ts.head()

Datetime
2012-08-25 00:00:00    8
2012-08-25 01:00:00    2
2012-08-25 02:00:00    6
2012-08-25 03:00:00    2
2012-08-25 04:00:00    2
Name: Count, dtype: int64

In [24]:
plt.figure(figsize = (16,8)) 
plt.plot(ts, label = 'Passenger Count')
plt.title('Time Series')
plt.xlabel("Time(year-month)")
plt.ylabel("Passenger count")
plt.legend(loc = 'best')
plt.savefig('Serie1.pdf')
plt.show()

<img src="Serie1.pdf" style="width:900px;height:300;">